# SAM

## Utils

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [ ]:
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))


## Models

In [ ]:
from PIL import Image
import requests
from transformers import SamModel, SamProcessor, pipeline

model_name = "facebook/sam-vit-huge"

start = time.time()
generator =  pipeline("mask-generation", device = 0, points_per_batch = 256, model=model_name)
end = time.time()
print(end-start)
# image_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
# outputs = generator(image_url, points_per_batch = 256)


# model = SamModel.from_pretrained("facebook/sam-vit-huge")
# processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

# img_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")
# input_points = [[[450, 600]]] # 2D localization of a window


In [ ]:
import time

In [ ]:
start = time.time()
generator =  pipeline("mask-generation", device = 0, points_per_batch = 4, model=model_name)
image_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
end = time.time()
end-start

In [ ]:
raw_image = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")

In [ ]:
start = time.time()
outputs = generator(image_url, points_per_batch = 4)
end = time.time()
end-start

In [ ]:
len(outputs["masks"])

In [ ]:
plt.imshow(np.array(raw_image))
ax = plt.gca()
for mask in outputs["masks"][:10]:
# mask = outputs["masks"][2]
    show_mask(mask, ax=ax, random_color=True)
plt.axis("off")
plt.show()

## Baseline Images

### All segmentations

In [ ]:
!unzip images.zip -d images

In [ ]:
import os

In [ ]:
# -------- Settings --------
input_folder = "images/images/"
output_folder = "sam_results"
os.makedirs(output_folder, exist_ok=True)

# -------- Process all images --------
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))]

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)

    start = time.time()
    outputs = generator(img_path, points_per_batch=4)
    end = time.time()

    print(f"Processed {img_name} in {end-start:.2f} seconds")

    # Load raw image
    raw_image = np.array(Image.open(img_path))

    # --- Plot result and save ----
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Left: original
    ax[0].imshow(raw_image)
    ax[0].set_title("Original")
    ax[0].axis("off")

    # Right: segmented
    ax[1].imshow(raw_image)
    for mask in outputs["masks"][:10]:  # limit masks
        show_mask(mask, ax=ax[1], random_color=True)
    ax[1].set_title("Segmented")
    ax[1].axis("off")

    save_path = os.path.join(output_folder, f"{os.path.splitext(img_name)[0]}_segmented.png")
    plt.savefig(save_path, bbox_inches="tight")
    plt.close()

In [ ]:
for files in os.listdir(output_folder):
    img = Image.open(os.path.join(output_folder, files))
    plt.imshow(img)
    plt.axis("off")
    plt.show()

### prompt

In [ ]:
# -------- Settings --------
input_folder = "images/images/"
output_folder = "sam_results"
os.makedirs(output_folder, exist_ok=True)

# -------- Process all images --------
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))]

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)

    start = time.time()
    outputs = generator(img_path, points_per_batch=4)
    end = time.time()

    print(f"Processed {img_name} in {end-start:.2f} seconds")

    # Load raw image
    raw_image = np.array(Image.open(img_path))

    # --- Plot result and save ----
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Left: original
    ax[0].imshow(raw_image)
    ax[0].set_title("Original")
    ax[0].axis("off")

    # Right: segmented
    ax[1].imshow(raw_image)
    for mask in outputs["masks"][:10]:  # limit masks
        show_mask(mask, ax=ax[1], random_color=True)
    ax[1].set_title("Segmented")
    ax[1].axis("off")

    save_path = os.path.join(output_folder, f"{os.path.splitext(img_name)[0]}_segmented.png")
    plt.savefig(save_path, bbox_inches="tight")
    plt.close()

In [ ]:
!zip -r -q sam_results.zip sam_results

## Quantization

For now it can't be done using bitsandbytes

In [ ]:
pip install -U -q bitsandbytes

In [ ]:
!zip -r -q sam_results_zip.zip sam_results

In [ ]:
from transformers import pipeline, BitsAndBytesConfig

model_name = "facebook/sam-vit-huge"


bnb_config = BitsAndBytesConfig(load_in_8bit=True)

pipe_8bit = pipeline(
    "mask-generation",
    model=model_name,
    model_kwargs={"quantization_config": bnb_config},
    points_per_batch=4,
)


In [ ]:
import time
import numpy as np
start = time.time()
image_url = "/content/images/images/chameleon.jpg"
img = Image.open(image_url)
# Run inference
outputs = pipe_8bit(img)

end = time.time()
print("Time taken:", end - start)

plt.imshow(np.array(raw_image))
ax = plt.gca()
for mask in outputs["masks"][:10]:
# mask = outputs["masks"][2]
    show_mask(mask, ax=ax, random_color=True)
plt.axis("off")
plt.show()

# SAM-HQ

In [ ]:
from transformers import SamHQModel, SamHQProcessor
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SamHQModel.from_pretrained("syscv-community/sam-hq-vit-base").to(device)
processor = SamHQProcessor.from_pretrained("syscv-community/sam-hq-vit-base")

In [ ]:
input_points = [[550, 200]]
input_label = np.array([1])
plt.imshow(raw_image)
show_points(np.array(input_points), input_label, plt.gca())

In [ ]:
inputs = processor(raw_image, input_points=[input_points], return_tensors="pt").to(device)
image_np = np.array(raw_image)
with torch.no_grad():
    outputs = model(**inputs, hq_token_only=True)

masks = processor.image_processor.post_process_masks(
    outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu()
)



In [ ]:
# You now have masks: list of boolean arrays (or floats). Save visual or mask files.
# Example: visualize original + mask overlay
plt.imshow(image_np)
ax = plt.gca()
for mask in masks:
    ax.imshow(mask[0,0,:,:], alpha=0.5)  # overlay
plt.axis("off")

In [ ]:
img_path = os.path.join(input_folder, image_files[0])
img_name = "giraffe"
start = time.time()
outputs = generator(img_path, points_per_batch=4)
end = time.time()

print(f"Processed {img_name} in {end-start:.2f} seconds")

In [ ]:
outputs['scores']

In [ ]:
ax = plt.gca()

# Left: original
# ax[0].imshow(raw_image)
# ax[0].set_title("Original")
# ax[0].axis("off")

# Right: segmented
plt.imshow(raw_image)
for mask in outputs["masks"][:10]:  # limit masks
    show_mask(mask, ax=ax, random_color=True)
plt.title("Segmented")
plt.axis("off")

In [ ]:
# -------- Settings --------
input_folder = "images/images/"
output_folder = "sam_hq_results"
os.makedirs(output_folder, exist_ok=True)

# -------- Process all images --------
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))]

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)

    start = time.time()
    outputs = generator(img_path, points_per_batch=4)
    end = time.time()

    print(f"Processed {img_name} in {end-start:.2f} seconds")

    # Load raw image
    raw_image = np.array(Image.open(img_path))

    # --- Plot result and save ----
    ax = plt.gca()

    # Left: original
    # ax[0].imshow(raw_image)
    # ax[0].set_title("Original")
    # ax[0].axis("off")

    # Right: segmented
    plt.imshow(raw_image)
    for mask in outputs["masks"][:10]:  # limit masks
        show_mask(mask, ax=ax, random_color=True)
    plt.set_title("Segmented")
    plt.axis("off")

    save_path = os.path.join(output_folder, f"{os.path.splitext(img_name)[0]}_segmented.png")
    plt.savefig(save_path, bbox_inches="tight")
    plt.close()

# Base lines Utils for ultralytics

In [ ]:
import os
import glob
import json
import time
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Ensure the required libraries are installed:
# pip install ultralytics opencv-python-headless matplotlib

def process_images_in_directory(model, input_dir, output_dir, timings_file, prompt_text="an object", use_prompt=False):
    """
    Loads all images from an input directory, runs the FastSAM model on them,
    stores inference timings in a JSON file, and saves the annotated results
    to an output directory.

    Args:
        model: The loaded FastSAM model.
        input_dir (str): Path to the directory containing input images.
        output_dir (str): Path to save the annotated images.
        timings_file (str): Path to save the JSON file with timings.
        prompt_text (str): The text prompt to guide the segmentation.
    """
    print(f"🚀 Starting image processing...")
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    timings = {}

    # Find all common image files
    image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tif", "*.tiff"]
    image_paths = []
    for ext in image_extensions:
        image_paths.extend(glob.glob(os.path.join(input_dir, ext)))

    if not image_paths:
        print(f"⚠️ No images found in {input_dir}. Exiting.")
        return

    print(f"Found {len(image_paths)} images to process.")

    for i, image_path in enumerate(image_paths):
        image_name = os.path.basename(image_path)
        print(f"Processing [{i+1}/{len(image_paths)}]: {image_name}")

        try:
            # --- Timing and Inference ---
            start_time = time.time()

            # Run the model
            if use_prompt:
                base_name, _ = os.path.splitext(image_name)
                results = model(image_path, device='cuda', texts=base_name)
            else:
                results = model(image_path, device='cuda') # Use 'cuda' if available

            end_time = time.time()
            # ------------------------------

            duration = end_time - start_time
            timings[image_name] = results[0].speed

            # Get the annotated image (as a NumPy array)
            annotated_img = results[0].plot()
            # plot_custom_masks(results[0], save_path=os.path.join(output_dir, image_name))

            # --- Save the Result ---
            output_path = os.path.join(output_dir, image_name)
            # cv2.imwrite saves in BGR format, which is what results[0].plot() provides
            cv2.imwrite(output_path, annotated_img)

        except Exception as e:
            print(f"❌ Error processing {image_name}: {e}")
            timings[image_name] = f"Error: {str(e)}"

    # --- Save Timings to JSON ---
    with open(timings_file, 'w') as f:
        json.dump(timings, f, indent=4)

    print(f"\n✅ Processing complete!")
    print(f"Annotated images saved to: {output_dir}")
    print(f"Inference timings saved to: {timings_file}")


def plot_side_by_side_results(input_dir, output_dir):
    """
    Finds corresponding images in an input and output directory
    and plots them side-by-side using matplotlib.

    Args:
        input_dir (str): Path to the directory with original images.
        output_dir (str): Path to the directory with annotated images.
    """
    print(f"\n📊 Plotting results...")

    # Find all image files in the output directory
    image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tif", "*.tiff"]
    annotated_paths = []
    for ext in image_extensions:
        annotated_paths.extend(glob.glob(os.path.join(output_dir, ext)))

    if not annotated_paths:
        print(f"⚠️ No annotated images found in {output_dir}. Cannot plot.")
        return

    for annotated_path in annotated_paths:
        image_name = os.path.basename(annotated_path)
        original_path = os.path.join(input_dir, image_name)

        # Check if the corresponding original image exists
        if not os.path.exists(original_path):
            print(f"Skipping {image_name}: Original image not found at {original_path}")
            continue

        # --- Load Images ---
        # cv2.imread loads as BGR
        original_img_bgr = cv2.imread(original_path)
        annotated_img_bgr = cv2.imread(annotated_path)

        # Convert BGR to RGB for matplotlib
        original_img_rgb = cv2.cvtColor(original_img_bgr, cv2.COLOR_BGR2RGB)
        annotated_img_rgb = cv2.cvtColor(annotated_img_bgr, cv2.COLOR_BGR2RGB)

        # --- Plotting ---
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))

        # Original Image
        axes[0].imshow(original_img_rgb)
        axes[0].set_title(f"Original: {image_name}")
        axes[0].axis("off")

        # Annotated Image
        axes[1].imshow(annotated_img_rgb)
        axes[1].set_title(f"Annotated: {image_name}")
        axes[1].axis("off")

        plt.tight_layout()
        plt.show()

In [ ]:
# results[0].masks.shape

In [ ]:
def plot_custom_masks(result, save_path="yup.png"):
    """
    Plots only the segmentation masks from a FastSAM result object,
    each with a unique random color. No boxes or text are displayed.

    Args:
        result: A single result object (e.g., results[0]) from a FastSAM model call.
    """

    # 1. Check if any masks were found
    if result.masks is None:
        print("No masks were found to plot.")
        return

    # 2. Get the raw mask data (PyTorch tensor)
    masks_data = result.masks.data

    # 3. Move tensor to CPU and convert to NumPy array
    # Shape becomes [N, H, W] where N is the number of masks
    masks_np = masks_data.cpu().numpy()

    # 4. Get the original image shape (height, width)
    # try:
    #     h, w = result.orig_shape
    # except AttributeError:
        # Fallback if orig_shape is not available (though it should be)
    h, w = masks_np.shape[1], masks_np.shape[2]

    # 5. Create an empty 4-channel (RGBA) image to draw all masks on
    # This acts as our canvas. It's initialized to [0,0,0,0] (transparent black)
    composite_mask_img = np.zeros((h, w, 4), dtype=np.uint8)

    num_masks = masks_np.shape[0]

    if num_masks == 0:
        print("Mask data is present, but 0 masks were found.")
        return

    # 6. Loop through each mask and apply a random color
    for i in range(num_masks):
        # Generate a unique random color (R, G, B)
        color_rgb = np.random.randint(0, 256, 3)
        # Create the full RGBA color, setting Alpha to 255 (opaque)
        color_rgba = np.array([color_rgb[0], color_rgb[1], color_rgb[2], 255], dtype=np.uint8)

        # Get the current mask (a 2D array of 0s and 1s)
        mask = masks_np[i]

        # 7. Apply the opaque color to our canvas
        # This uses boolean indexing:
        # "wherever the mask is 1 (or > 0), set the pixel in composite_mask_img to this color"
        composite_mask_img[mask > 0] = color_rgba

    # 8. Display the final image (overlayed)

    # 8a. Get original image and convert BGR to RGB
    # (ultralytics/cv2 loads as BGR, matplotlib expects RGB)
    original_img_rgb = result.orig_img[..., ::-1]

    plt.figure(figsize=(8, 8))

    # 8b. Plot the original image as the base layer
    plt.imshow(original_img_rgb)

    # 8c. Plot the mask "canvas" on top.
    # The transparent [0,0,0,0] pixels will let the original image show through.
    plt.imshow(composite_mask_img.astype(np.uint8), alpha=0.7)

    plt.title(f"Found {num_masks} masks (Overlayed on Original)")
    plt.axis("off")
    plt.savefig(save_path)
    plt.show()

# Fast SAM using ultralytics

In [ ]:
!pip install -q ultralytics

In [ ]:
from ultralytics import FastSAM
import time

# Define an inference source
source = "/content/images/images/chameleon.jpg"

start = time.time()
# Create a FastSAM model
model = FastSAM("FastSAM-s.pt")  # or FastSAM-x.pt
end = time.time()
print(start-end)

# Run inference on an image
# everything_results = model(source, device=device, retina_masks=True, imgsz=1024, conf=0.4, iou=0.9)

# # Run inference with bboxes prompt
# results = model(source, bboxes=[439, 437, 524, 709])

# # Run inference with points prompt
# results = model(source, points=[[200, 200]], labels=[1])

# # Run inference with texts prompt
# results = model(source, texts="a photo of a dog")

In [ ]:
annotated_img = everything_results[0].plot()

plt.figure(figsize=(8,8))
plt.imshow(annotated_img)
plt.axis("off")
plt.show()

In [ ]:
plot_custom_masks(results[0])

In [ ]:
source = "/content/images/images/giraffe_spot.jpg"
results = model(source)
plot_custom_masks(results[0])


In [ ]:
# Run inference with points prompt
results = model(source, points=[[300, 200]], labels=[1])
annotated_img = results[0].plot()

plt.figure(figsize=(8,8))
plt.imshow(annotated_img)
plt.axis("off")
plt.show()

In [ ]:
source = "/content/images/images/giraffe_spot.jpg"
results = model(source, texts="a giraffe")
annotated_img = results[0].plot()

plt.figure(figsize=(8,8))
plt.imshow(annotated_img)
plt.axis("off")
plt.show()

In [ ]:
source = "/content/images/images/owl_spot-min.png"
results = model(source, texts="an owl")
annotated_img = results[0].plot()

plt.figure(figsize=(8,8))
plt.imshow(annotated_img)
plt.axis("off")
plt.show()

## Base line images

In [ ]:
INPUT_DIR = "images"
OUTPUT_DIR = "fast_sam_prompts"
TIMINGS_FILE = "fastsam_timings.json"

PROMPT_TEXT = "a person" # <-- !!! CHANGE THIS to match your images

# 3. Create dummy input directory (if it doesn't exist)
#    You MUST add your own images to this folder!
os.makedirs(INPUT_DIR, exist_ok=True)

# --- Run Processing ---
process_images_in_directory(
    model=model,
    input_dir=INPUT_DIR,
    output_dir=OUTPUT_DIR,
    timings_file=TIMINGS_FILE,
    prompt_text=PROMPT_TEXT,
    # use_prompt=True
)


In [ ]:
OUTPUT_DIR = "fast_sam_results"
# --- Plot Results ---
plot_side_by_side_results(INPUT_DIR, OUTPUT_DIR)

### prompts

In [ ]:
INPUT_DIR = "images"
OUTPUT_DIR = "fast_sam_prompt_results"
TIMINGS_FILE = "fastsam_prompt_timings.json"

PROMPT_TEXT = "a person" # <-- !!! CHANGE THIS to match your images

# 3. Create dummy input directory (if it doesn't exist)
#    You MUST add your own images to this folder!
os.makedirs(INPUT_DIR, exist_ok=True)

# --- Run Processing ---
process_images_in_directory(
    model=model,
    input_dir=INPUT_DIR,
    output_dir=OUTPUT_DIR,
    timings_file=TIMINGS_FILE,
    prompt_text=PROMPT_TEXT,
    use_prompt=True
)


In [ ]:
# --- Plot Results ---
plot_side_by_side_results(INPUT_DIR, OUTPUT_DIR)

In [ ]:
!zip -r -q fast_sam_prompt_results.zip fast_sam_prompt_results

# SAM2 using ultralytics

## Base Model

In [ ]:
from ultralytics import SAM

start = time.time()
# Load a model
model = SAM("sam2.1_b.pt")
end = time.time()
end-start

In [ ]:
source = "/content/images/images/chameleon.jpg"
results = model(source)
annotated_img = results[0].plot()

plt.figure(figsize=(8,8))
plt.imshow(annotated_img)
plt.axis("off")
plt.show()

In [ ]:
results = model(source, points=[500, 200], labels=[1])

annotated_img = results[0].plot()

plt.figure(figsize=(8,8))
plt.imshow(annotated_img)
plt.axis("off")
plt.show()

In [ ]:
!zip -r -q fast_sam_results.zip fast_sam_results

### Baseline images

In [ ]:
INPUT_DIR = "images/images"
OUTPUT_DIR = "sam2_base_results1"
TIMINGS_FILE = "sam2_base_timings.json"

PROMPT_TEXT = "a person" # <-- !!! CHANGE THIS to match your images

# 3. Create dummy input directory (if it doesn't exist)
#    You MUST add your own images to this folder!
os.makedirs(INPUT_DIR, exist_ok=True)

# --- Run Processing ---
process_images_in_directory(
    model=model,
    input_dir=INPUT_DIR,
    output_dir=OUTPUT_DIR,
    timings_file=TIMINGS_FILE,
    prompt_text=PROMPT_TEXT
)

plot_side_by_side_results(INPUT_DIR, OUTPUT_DIR)


In [ ]:
!zip -r -q sam2_base_results.zip sam2_base_results1

## Large Model

In [ ]:
from ultralytics import SAM
start = time.time()
# Load a model
model = SAM("sam2.1_l.pt")
end = time.time()
end-start

In [ ]:
INPUT_DIR = "images/images"
OUTPUT_DIR = "sam2_large_results"
TIMINGS_FILE = "sam2_large_timings.json"

PROMPT_TEXT = "a person" # <-- !!! CHANGE THIS to match your images

# 3. Create dummy input directory (if it doesn't exist)
#    You MUST add your own images to this folder!
os.makedirs(INPUT_DIR, exist_ok=True)

# --- Run Processing ---
process_images_in_directory(
    model=model,
    input_dir=INPUT_DIR,
    output_dir=OUTPUT_DIR,
    timings_file=TIMINGS_FILE,
    prompt_text=PROMPT_TEXT
)

plot_side_by_side_results(INPUT_DIR, OUTPUT_DIR)


In [ ]:
!zip -r -q sam2_large_results.zip sam2_large_results

# Other Models

In [ ]:
del model

In [ ]:
!pip install -U -q segmentation_models_pytorch albumentations

In [ ]:
import torch
import requests
import numpy as np
import albumentations as A
import segmentation_models_pytorch as smp

from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load pretrained model and preprocessing function
checkpoint = "smp-hub/segformer-b5-640x640-ade-160k"
model = smp.from_pretrained(checkpoint).eval().to(device)
preprocessing = A.Compose.from_pretrained(checkpoint)


In [ ]:

# Load image
url = "/content/images/images/chameleon.jpg"
image = Image.open(url)

# Preprocess image
np_image = np.array(image)
normalized_image = preprocessing(image=np_image)["image"]
input_tensor = torch.as_tensor(normalized_image)
input_tensor = input_tensor.permute(2, 0, 1).unsqueeze(0)  # HWC -> BCHW
input_tensor = input_tensor.to(device)

# Perform inference
with torch.no_grad():
    output_mask = model(input_tensor)

# Postprocess mask
mask = torch.nn.functional.interpolate(
    output_mask, size=(image.height, image.width), mode="bilinear", align_corners=False
)
mask = mask.argmax(1).cpu().numpy()  # argmax over predicted classes (channels dim)


In [ ]:
plt.imshow(image)
plt.imshow(mask[0,:,:], alpha=0.9)

# finegrain/finegrain-box-segmenter

## utils

In [ ]:
!pip install -q pymatting

In [ ]:
# 1. Clone using HTTPS (not SSH)
!git clone -q https://github.com/finegrain-ai/refiners.git

# 2. Move into the directory
%cd refiners

!pip install -e .[all]

In [ ]:
from PIL import Image
import time
from refiners.solutions import BoxSegmenter


In [ ]:
import time
from collections.abc import Sequence
from typing import Any, cast

import requests
import torch
import numpy as np
from PIL import Image
from pymatting.foreground.estimate_foreground_ml import estimate_foreground_ml
from refiners.fluxion.utils import no_grad
from refiners.solutions import BoxSegmenter
from transformers import GroundingDinoForObjectDetection, GroundingDinoProcessor

# Define the bounding box type hint for clarity
BoundingBox = tuple[int, int, int, int]

# --- Model Loading ---

print("Setting up device...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Loading Finegrain BoxSegmenter model...")
# Load segmenter on CPU first, then move model to GPU (as in the space code)
segmenter = BoxSegmenter(device="cpu")
segmenter.device = device
segmenter.model = segmenter.model.to(device=segmenter.device)

print("Loading GroundingDINO model...")
gd_model_path = "IDEA-Research/grounding-dino-base"
gd_processor = GroundingDinoProcessor.from_pretrained(gd_model_path)
gd_model = GroundingDinoForObjectDetection.from_pretrained(gd_model_path, torch_dtype=torch.float32)
gd_model = gd_model.to(device=device)
assert isinstance(gd_model, GroundingDinoForObjectDetection)

print("Models loaded successfully.")

# --- Helper Functions (copied from the Space) ---

def bbox_union(bboxes: Sequence[list[int]]) -> BoundingBox | None:
    """Calculates the union of a list of bounding boxes."""
    if not bboxes:
        return None
    for bbox in bboxes:
        assert len(bbox) == 4
        assert all(isinstance(x, int) for x in bbox)
    return (
        min(bbox[0] for bbox in bboxes),
        min(bbox[1] for bbox in bboxes),
        max(bbox[2] for bbox in bboxes),
        max(bbox[3] for bbox in bboxes),
    )


def corners_to_pixels_format(bboxes: torch.Tensor, width: int, height: int) -> torch.Tensor:
    """Converts bounding box format and clamps to image dimensions."""
    x1, y1, x2, y2 = bboxes.round().to(torch.int32).unbind(-1)
    return torch.stack((x1.clamp_(0, width), y1.clamp_(0, height), x2.clamp_(0, width), y2.clamp_(0, height)), dim=-1)


def gd_detect(img: Image.Image, prompt: str) -> BoundingBox | None:
    """
    Detects objects in an image based on a text prompt using GroundingDINO.
    Returns a single bounding box that is the union of all detections.
    """
    print(f"Running GroundingDINO with prompt: '{prompt}'")
    assert isinstance(gd_processor, GroundingDinoProcessor)

    # Grounding Dino expects a dot after each category.
    inputs = gd_processor(images=img, text=f"{prompt}.", return_tensors="pt").to(device=device)

    with no_grad():
        outputs = gd_model(**inputs)

    width, height = img.size
    results: dict[str, Any] = gd_processor.post_process_grounded_object_detection(
        outputs,
        inputs["input_ids"],  # type: ignore
        target_sizes=[(height, width)],
    )[0]
    assert "boxes" in results and isinstance(results["boxes"], torch.Tensor)

    bboxes = corners_to_pixels_format(results["boxes"].cpu(), width, height)
    return bbox_union(bboxes.numpy().tolist())


def apply_mask_to_image(
    img: Image.Image,
    mask_img: Image.Image,
    defringe: bool = True,
) -> Image.Image:
    """Applies the mask to the original image to create a cutout."""
    assert img.size == mask_img.size
    img = img.convert("RGB")
    mask_img = mask_img.convert("L")

    if defringe:
        print("Applying defringe (color decontamination)...")
        # Mitigate edge halo effects via color decontamination
        rgb, alpha = np.asarray(img) / 255.0, np.asarray(mask_img) / 255.0
        foreground = cast(np.ndarray[Any, np.dtype[np.uint8]], estimate_foreground_ml(rgb, alpha))
        img = Image.fromarray((foreground * 255).astype("uint8"))

    result = Image.new("RGBA", img.size)
    result.paste(img, (0, 0), mask_img)
    return result


In [ ]:
import numpy as np
from PIL import Image
from pymatting.alpha.estimate_alpha_cf import estimate_alpha_cf
from pymatting.util.util import stack_images

def _create_trimap_from_mask(mask_np, low_thresh=0.1, high_thresh=0.9):
    """
    Creates a trimap from a semi-transparent mask.

    Values: 0.0 = Definite BG, 0.5 = Unknown, 1.0 = Definite FG
    """
    trimap = np.full(mask_np.shape, 0.5)  # Start with all unknown
    trimap[mask_np < low_thresh] = 0.0  # Definite Background
    trimap[mask_np > high_thresh] = 1.0 # Definite Foreground
    return trimap

def refine_mask_with_matting(image: Image.Image, initial_mask: Image.Image) -> Image.Image:
    """
    Refines a given mask using the Closed-Form Matting algorithm.

    :param image: The original RGB image.
    :param initial_mask: The "good" but imperfect mask from BoxSegmenter (as a PIL Image).
    :return: A new, refined PIL Image mask.
    """
    print("Refining mask with Closed-Form Matting...")

    # 1. Convert PIL Images to numpy arrays (0.0 - 1.0 range)
    image_np = np.asarray(image.convert("RGB")) / 255.0
    mask_np = np.asarray(initial_mask.convert("L")) / 255.0

    # 2. Create a trimap from our initial mask
    # This defines the "unknown" regions for the matting algorithm to solve
    trimap = _create_trimap_from_mask(mask_np)

    # 3. Call the matting algorithm
    # This is the magic step that solves for the new alpha values
    # in the "unknown" region.
    new_alpha_np = estimate_alpha_cf(image_np, trimap)

    # 4. Convert the refined numpy alpha mask back to a PIL Image
    new_mask_pil = Image.fromarray((new_alpha_np * 255).astype(np.uint8), 'L')

    print("Mask refinement complete.")
    return new_mask_pil

In [ ]:
def show_mask(mask, ax, random_color=False):
    """
    Overlays a single segmentation mask on a Matplotlib axis.

    Parameters:
    - mask (np.array): A 2D boolean or binary (0/1) mask.
    - ax (matplotlib.axes.Axes): The axis to plot on.
    - random_color (bool): If True, a random color is used for the mask.
    """
    if random_color:
        # Generate random color (R, G, B) + set alpha to 0.6
        color = np.concatenate([np.random.rand(3), np.array([0.6])])
    else:
        # Use a default fixed color (blue) + alpha
        color = np.array([0.2, 0.5, 0.8, 0.6]) # RGBA

    # Create an empty RGBA image of the same size as the mask
    h, w = mask.shape[-2:]
    mask_image = np.zeros((h, w, 4))

    # Where the mask is True (or > 0), set the color
    mask_image[mask.astype(bool)] = color

    # Overlay the mask image on the existing plot
    ax.imshow(mask_image)

## playing with model

In [ ]:
# 1. Load your image
image_url = "https://huggingface.co/spaces/finegrain/finegrain-object-cutter/resolve/main/examples/potted-plant.jpg"
img = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")

# 2. Define your text prompt
text_prompt = "potted plant"

start_time = time.time()

# 3. Step 1: Get bounding box from GroundingDINO
bbox = gd_detect(img, text_prompt)
end_time = time.time()
print(f"Time for detecting box {end_time-start_time}")

In [ ]:
bbox

In [ ]:
start_time = time.time()
if not bbox:
    print(f"Error: No object detected for prompt '{text_prompt}'")
else:
    print(f"Detected bounding box: {bbox}")
    # 4. Step 2: Get mask from BoxSegmenter
    print("Running Finegrain BoxSegmenter...")
    mask = segmenter(img, box_prompt=bbox)

    # 5. Save the raw mask
    mask.save("output_raw_mask.png")
    print("Saved raw mask to 'output_raw_mask.png'")

    # 6. (Optional) Apply mask to image to create cutout
    cutout = apply_mask_to_image(img, mask, defringe=True)
    cutout.save("output_cutout.png")
    print("Saved final cutout to 'output_cutout.png'")

end_time = time.time()
print(f"Total time taken: {end_time - start_time:.2f} seconds")

In [ ]:
device = torch.device('cuda')
device

In [ ]:


start = time.time()
# Initialize the segmenter (downloads weights)
segmenter = BoxSegmenter(device=device)
end = time.time()
print(end-start)

In [ ]:
input_image.width

In [ ]:
# Load your image
source = "images/llama_img.jpg"
input_image = Image.open(source)

# Define your bounding box
# Format is (x_min, y_min, x_max, y_max)
box = (0, 0, input_image.width, input_image.height)

# Generate the mask
start = time.time()
mask = segmenter(input_image, box_prompt=box)
end = time.time()
print(end-start)

# Save the output mask
# mask.save("output_mask.png")

In [ ]:
np.array(mask).shape

In [ ]:
np.array(input_image).shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(np.array(input_image))
# ax = plt.gca()
# for mask in outputs["masks"][:10]:
# mask = outputs["masks"][2]
# show_mask(np.array(mask), ax=ax, random_color=True)
plt.imshow(mask, alpha=0.6)
plt.axis("off")
plt.show()

In [ ]:
mask.save("llama_mask.png")

In [ ]:
source = "llama_img.jpg"
input_image = Image.open(source)

# Define your bounding box
# Format is (x_min, y_min, x_max, y_max)
box = (0, 0, input_image.width, input_image.height)

# Generate the mask
start = time.time()
mask = segmenter(input_image, box_prompt=box)
end = time.time()
print(end-start)

In [ ]:
refined_mask = refine_mask_with_matting(input_image, mask)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(np.array(input_image))
ax = plt.gca()
# for mask in outputs["masks"][:10]:
# mask = outputs["masks"][2]
show_mask(np.array(refined_mask), ax=ax, random_color=True)
plt.axis("off")
plt.show()

In [ ]:
def apply_mask(
    img: Image.Image,
    mask_img: Image.Image,
    defringe: bool = True,
) -> Image.Image:
    assert img.size == mask_img.size
    img = img.convert("RGB")
    mask_img = mask_img.convert("L")

    if defringe:
        # Mitigate edge halo effects via color decontamination
        rgb, alpha = np.asarray(img) / 255.0, np.asarray(mask_img) / 255.0
        foreground = cast(np.ndarray[Any, np.dtype[np.uint8]], estimate_foreground_ml(rgb, alpha))
        img = Image.fromarray((foreground * 255).astype("uint8"))

    result = Image.new("RGBA", img.size)
    result.paste(img, (0, 0), mask_img)
    return result

In [ ]:
result_img = apply_mask(input_image, mask)

In [ ]:
result_img

## baselines images

### box

In [ ]:
!unzip -q images.zip

In [ ]:
img_path = "/content/images/chameleon.jpg"
filename =  os.path.basename(img_path)
filename

In [ ]:
from tqdm import tqdm
import glob

def process_directory(input_dir: str, output_dir: str):
    """
    Processes all images in an input directory and saves the
    refined mask overlays and a timing JSON in the output directory.
    """
    print(f"Starting processing for directory: {input_dir}")
    os.makedirs(output_dir, exist_ok=True)

    # 1. Find all valid images
    valid_extensions = ('*.jpg', '*.jpeg', '*.png')
    image_paths = []
    for ext in valid_extensions:
        image_paths.extend(glob.glob(os.path.join(input_dir, ext)))

    if not image_paths:
        print(f"No images found in {input_dir}. Exiting.")
        return

    print(f"Found {len(image_paths)} images to process.")

    timings = {} # Dictionary to store processing times

    # 2. Process each image
    for img_path in tqdm(image_paths, desc="Processing images"):
        filename = os.path.basename(img_path)
        base_name, _ = os.path.splitext(filename)
        output_plot_path = os.path.join(output_dir, f"{filename}")

        try:
            # --- Start of your single-image logic ---
            input_image = Image.open(img_path).convert("RGB") # Ensure RGB

            # 1. Get initial mask (using full image as box)
            box = (0, 0, input_image.width, input_image.height)

            start_segment = time.time()
            mask = segmenter(input_image, box_prompt=box)
            segment_time = time.time() - start_segment

            # # 2. Refine the mask
            # start_refine = time.time()
            # refined_mask = refine_mask_with_matting(input_image, mask)
            # refine_time = time.time() - start_refine

            # total_time = segment_time + refine_time

            # 3. Store timing info
            timings[filename] = round(segment_time, 2)
                # "segment_time": round(segment_time, 2),
                # "refine_time": round(refine_time, 2),
                # "total_time": round(total_time, 2),
                # "status": "Success"


            # 4. Plot and save the result
            plt.figure(figsize=(10, 7)) # Create a new figure
            plt.imshow(np.array(input_image))
            ax = plt.gca()
            show_mask(np.array(mask), ax=ax, random_color=True)
            # plt.imshow(mask, alpha=0.6)
            plt.axis("off")

            # Save the figure, removing white borders
            plt.savefig(output_plot_path, bbox_inches='tight', pad_inches=0)

            # Close the figure to free up memory
            plt.close()

        except Exception as e:
            print(f"\n[ERROR] Failed to process {filename}: {e}")
            timings[filename] = {"status": "Error", "message": str(e)}
            # Continue to the next image

    # 3. Save the timings JSON file
    json_path = os.path.join(output_dir, "processing_timings.json")
    with open(json_path, 'w') as f:
        json.dump(timings, f, indent=4)

    print(f"\nProcessing complete.")
    print(f"Results saved to: {output_dir}")
    print(f"Timing data saved to: {json_path}")

In [ ]:
import os
import matplotlib.pyplot as plt
import warnings
import json
warnings.filterwarnings("ignore")

In [ ]:
process_directory("images","finegrain_results2")

In [ ]:
# Without matting

plot_side_by_side_results("images","finegrain_results2")

In [ ]:
# After matting

plot_side_by_side_results("images","finegrain_results3")

In [ ]:
!zip -r -q finegrain_results_after_matting.zip finegrain_results3

### prompt

In [ ]:
from tqdm import tqdm
import glob

def process_directory(input_dir: str, output_dir: str):
    """
    Processes all images in an input directory and saves the
    refined mask overlays and a timing JSON in the output directory.
    """
    print(f"Starting processing for directory: {input_dir}")
    os.makedirs(output_dir, exist_ok=True)

    # 1. Find all valid images
    valid_extensions = ('*.jpg', '*.jpeg', '*.png')
    image_paths = []
    for ext in valid_extensions:
        image_paths.extend(glob.glob(os.path.join(input_dir, ext)))

    if not image_paths:
        print(f"No images found in {input_dir}. Exiting.")
        return

    print(f"Found {len(image_paths)} images to process.")

    timings = {} # Dictionary to store processing times

    # 2. Process each image
    for img_path in tqdm(image_paths, desc="Processing images"):
        filename = os.path.basename(img_path)
        base_name, _ = os.path.splitext(filename)
        output_plot_path = os.path.join(output_dir, f"{filename}")

        try:
            # --- Start of your single-image logic ---
            input_image = Image.open(img_path).convert("RGB") # Ensure RGB

            # 1. Get initial mask (using full image as box)
            box = (0, 0, input_image.width, input_image.height)

            # Auto generate box using another model
            text_prompt = base_name
            start_box = time.time()
            bbox = gd_detect(input_image, text_prompt)
            box_time = time.time() -start_box

            start_segment = time.time()
            mask = segmenter(input_image, box_prompt=bbox)
            segment_time = time.time() - start_segment

            # # 2. Refine the mask
            # start_refine = time.time()
            # refined_mask = refine_mask_with_matting(input_image, mask)
            # refine_time = time.time() - start_refine

            total_time = segment_time + box_time

            # 3. Store timing info
            timings[filename] = {
                "segment_time": round(segment_time, 2),
                "box_time": round(box_time, 2),
                "total_time": round(total_time, 2),
                # "status": "Success"
            }

            # 4. Plot and save the result
            plt.figure(figsize=(10, 7)) # Create a new figure
            plt.imshow(np.array(input_image))
            ax = plt.gca()
            show_mask(np.array(mask), ax=ax, random_color=True)
            # plt.imshow(mask, alpha=0.6)
            plt.axis("off")

            # Save the figure, removing white borders
            plt.savefig(output_plot_path, bbox_inches='tight', pad_inches=0)

            # Close the figure to free up memory
            plt.close()

        except Exception as e:
            print(f"\n[ERROR] Failed to process {filename}: {e}")
            timings[filename] = {"status": "Error", "message": str(e)}
            # Continue to the next image

    # 3. Save the timings JSON file
    json_path = os.path.join(output_dir, "processing_timings.json")
    with open(json_path, 'w') as f:
        json.dump(timings, f, indent=4)

    print(f"\nProcessing complete.")
    print(f"Results saved to: {output_dir}")
    print(f"Timing data saved to: {json_path}")

In [ ]:
process_directory("images","finegrain_results_prompt")

In [ ]:
# Without matting

plot_side_by_side_results("images","finegrain_results_prompt")